In [37]:
import pandas as  pd 
import numpy as np 
import sqlite3 

In [38]:
dataset = pd.read_csv(r"C:\Users\ayush\Downloads\WA_Fn-UseC_-Telco-Customer-Churn (1).csv")
dataset.head(5)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [39]:
dataset.shape

(7043, 21)

In [40]:
connection = sqlite3.connect("Churn_database.db" , timeout = 30 )
cursor = connection.cursor()

In [41]:
dataset.to_sql(
    name="churn_data",
    con=connection,
    if_exists="replace",
    index=False
)

7043

In [42]:
querry = "SELECT * FROM churn_data limit 5 ;"
pd.read_sql(querry , connection)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [43]:
pd.read_sql("""
SELECT 
    SUM (CASE WHEN TotalCharges IS NULL OR TotalCharges = '' THEN 1 ELSE 0 END ) AS Missing_total_charges,
    
    SUM (CASE WHEN MonthlyCharges IS NULL OR MonthlyCharges = '' THEN 1 ELSE 0 END ) AS Missing_monthly_charges,

    SUM (CASE WHEN Contract IS NULL OR Contract = '' THEN 1 ELSE 0 END ) AS missing_Contract_values
    FROM churn_data;
""" , connection)

,Missing_total_charges,Missing_monthly_charges,missing_Contract_values
0,0,0,0


In [44]:
## Gender distribution  count 

pd.read_sql("""

SELECT gender , COUNT(*) AS count FROM churn_data 
GROUP BY gender  ;

""" , connection)

,gender,count
0,Female,3488
1,Male,3555


In [45]:
## Contract Type 

pd.read_sql("""

SELECT Contract , COUNT(*) AS contract_Type_count FROM churn_data
GROUP BY Contract ;
""" , connection)

,Contract,contract_Type_count
0,Month-to-month,3875
1,One year,1473
2,Two year,1695


In [46]:
## internet Service 

pd.read_sql("""

SELECT InternetService , COUNT(*) AS service_count FROM churn_data
GROUP BY InternetService ;
""" , connection)

,InternetService,service_count
0,DSL,2421
1,Fiber optic,3096
2,No,1526


In [47]:
pd.read_sql("""
SELECT Contract , COUNT(*) AS Total ,
    SUM (CASE WHEN Churn = 'Yes' OR Churn = 'yes' THEN 1 ELSE 0 END ) AS churned,
    ROUND(100.0 *SUM (CASE WHEN Churn = 'Yes' OR Churn = 'yes' THEN 1 ELSE 0 END )/COUNT(*) , 2) AS Churn_Rate
FROM churn_data 
GROUP BY Contract;

""" , connection)

,Contract,Total,churned,Churn_Rate
0,Month-to-month,3875,1655,42.71
1,One year,1473,166,11.27
2,Two year,1695,48,2.83


In [48]:
pd.read_sql("""
SELECT 
    InternetService, COUNT(*) AS Total ,SUM (CASE WHEN Churn = 'Yes' OR Churn = 'yes' THEN 1 ELSE 0 END ) AS churned,
    ROUND(100.0 *SUM (CASE WHEN Churn = 'Yes' OR Churn = 'yes' THEN 1 ELSE 0 END )/COUNT(*) , 2) AS Churn_Rate
FROM churn_data 
GROUP BY InternetService;

""" , connection)

,InternetService,Total,churned,Churn_Rate
0,DSL,2421,459,18.96
1,Fiber optic,3096,1297,41.89
2,No,1526,113,7.40


In [49]:
pd.read_sql("""
SELECT 
    PaymentMethod, COUNT(*) AS Total ,SUM (CASE WHEN Churn = 'Yes' OR Churn = 'yes' THEN 1 ELSE 0 END ) AS churned,
    ROUND(100.0 *SUM (CASE WHEN Churn = 'Yes' OR Churn = 'yes' THEN 1 ELSE 0 END )/COUNT(*) , 2) AS Churn_Rate
FROM churn_data 
GROUP BY PaymentMethod;

""" , connection)

,PaymentMethod,Total,churned,Churn_Rate
0,Bank transfer (automatic),1544,258,16.71
1,Credit card (automatic),1522,232,15.24
2,Electronic check,2365,1071,45.29
3,Mailed check,1612,308,19.11


In [50]:
pd.read_sql("""

SELECT CASE WHEN tenure  <= 6 THEN '0 - 6 months'
            WHEN tenure  <= 12 THEN '6 - 12 months'
            WHEN tenure  <= 24 THEN '1 - 2 years'
            ELSE '+ 2 years'
            END AS tenure_bucket ,
            COUNT(*) AS Total_customer , 
            ROUND(100.0 *SUM (CASE WHEN Churn = 'Yes' OR Churn = 'yes' THEN 1 ELSE 0 END )/COUNT(*) , 2) AS Churn_Rate
FROM churn_data
GROUP BY tenure_bucket
ORDER BY Churn_Rate

""", connection)

,tenure_bucket,Total_customer,Churn_Rate
0,+ 2 years,3833,14.04
1,1 - 2 years,1024,28.71
2,6 - 12 months,705,35.89
3,0 - 6 months,1481,52.94


In [51]:

cursor.execute("""
ALTER TABLE churn_data ADD COLUMN Tenure_bucket TEXT
""")

cursor.execute("""

UPDATE churn_data SET Tenure_bucket = CASE 
                                    WHEN tenure  <= 6 THEN 'new'
                                    WHEN tenure  <= 12 THEN 'Mid'
                                    WHEN tenure  <= 24 THEN 'long'
                                    ELSE 'loyal' END 
                """)
connection.commit()

In [52]:
cursor.execute("""
ALTER TABLE churn_data ADD COLUMN avg_monthly_spend REAL;
""")

cursor.execute("""
UPDATE churn_data SET TotalCharges = NULL 
WHERE TRIM(TotalCharges) = '';
""")

cursor.execute("""
UPDATE churn_data SET avg_monthly_spend = ROUND(COALESCE(CAST(TotalCharges AS REAL) / NULLIF(tenure, 0),MonthlyCharges),2);
""")
connection.cursor()

In [53]:
cursor.execute("""
ALTER TABLE churn_data ADD COLUMN Early_high_risk REAL ;
""")

cursor.execute ("""

UPDATE churn_data SET Early_high_risk  = CASE 
    WHEN tenure <= 6 AND MonthlyCharges > (SELECT AVG(MonthlyCharges) FROM churn_data )THEN 1 ELSE 0 END ;
""")
connection.cursor()

In [54]:
cursor.execute("""
ALTER TABLE churn_data ADD COLUMN life_time_risk_score ; 
""" )
cursor.execute("""

UPDATE churn_data 
    SET life_time_risk_score = ((CASE WHEN tenure <= 6 THEN 2 ELSE 1 END) *
                                (CASE WHEN Contract = 'Month-to-month' THEN 2 ELSE 1 END) * 
                                (CASE WHEN PaymentMethod = 'Electronic check' THEN 2 ELSE 1 END))/8.0 ;
""")
connection.cursor()

In [55]:
cursor.execute("""
ALTER TABLE churn_data ADD COLUMN Service_count;
""")
connection.cursor()

In [56]:
cursor.execute("""
UPDATE churn_data
SET Service_count =
    (CASE WHEN PhoneService = 'Yes' THEN 1 ELSE 0 END +
     CASE WHEN InternetService != 'No' THEN 1 ELSE 0 END +
     CASE WHEN OnlineSecurity = 'Yes' THEN 1 ELSE 0 END +
     CASE WHEN OnlineBackup = 'Yes' THEN 1 ELSE 0 END +
     CASE WHEN DeviceProtection = 'Yes' THEN 1 ELSE 0 END +
     CASE WHEN TechSupport = 'Yes' THEN 1 ELSE 0 END +
     CASE WHEN StreamingTV = 'Yes' THEN 1 ELSE 0 END +
     CASE WHEN StreamingMovies = 'Yes' THEN 1 ELSE 0 END);

""")
connection.cursor()

In [57]:
cursor.execute("""
ALTER TABLE churn_data ADD COLUMN Price_shook;
""")
connection.cursor()

In [58]:
cursor.execute("""
UPDATE churn_data
SET Price_shook = ROUND(MonthlyCharges - avg_monthly_spend, 2);
""")
connection.cursor()

In [59]:
cursor.execute("""
ALTER TABLE churn_data ADD COLUMN customer_value REAL;
""")
connection.cursor()

In [60]:
cursor.execute("""
UPDATE churn_data
SET customer_value = ROUND((tenure * MonthlyCharges * service_count) / 1000, 2);
""")
connection.cursor()

In [61]:
pd.read_sql("""
SELECT * FROM churn_data limit 5 ; 
""", connection)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,Tenure_bucket,avg_monthly_spend,Early_high_risk,life_time_risk_score,Service_count,Price_shook,customer_value
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No,new,29.85,0.0,1.000,2,0.00,0.06
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No,loyal,55.57,0.0,0.125,4,1.38,7.75
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,new,54.08,0.0,0.500,4,-0.23,0.43
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No,loyal,40.91,0.0,0.125,4,1.39,7.61
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,new,75.83,1.0,1.000,2,-5.13,0.28


In [62]:
pd.read_sql("PRAGMA table_info(churn_data);", connection)

,cid,name,type,notnull,dflt_value,pk
0,0,customerID,TEXT,0,None,0
1,1,gender,TEXT,0,None,0
2,2,SeniorCitizen,INTEGER,0,None,0
3,3,Partner,TEXT,0,None,0
4,4,Dependents,TEXT,0,None,0
5,5,tenure,INTEGER,0,None,0
6,6,PhoneService,TEXT,0,None,0
7,7,MultipleLines,TEXT,0,None,0
8,8,InternetService,TEXT,0,None,0
9,9,OnlineSecurity,TEXT,0,None,0


In [63]:
pd.set_option('display.max_columns', None)


In [64]:
pd.read_sql("SELECT * FROM churn_data LIMIT 5;", connection)


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,Tenure_bucket,avg_monthly_spend,Early_high_risk,life_time_risk_score,Service_count,Price_shook,customer_value
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No,new,29.85,0.0,1.000,2,0.00,0.06
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No,loyal,55.57,0.0,0.125,4,1.38,7.75
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,new,54.08,0.0,0.500,4,-0.23,0.43
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No,loyal,40.91,0.0,0.125,4,1.39,7.61
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,new,75.83,1.0,1.000,2,-5.13,0.28


In [65]:
processed_data = pd.read_sql(
    "SELECT * FROM churn_data;",
    connection
)

processed_data.columns


Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn',
       'Tenure_bucket', 'avg_monthly_spend', 'Early_high_risk',
       'life_time_risk_score', 'Service_count', 'Price_shook',
       'customer_value'],
      dtype='object')

In [66]:
processed_data.head(3)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,Tenure_bucket,avg_monthly_spend,Early_high_risk,life_time_risk_score,Service_count,Price_shook,customer_value
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No,new,29.85,0.0,1.000,2,0.00,0.06
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No,loyal,55.57,0.0,0.125,4,1.38,7.75
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,new,54.08,0.0,0.500,4,-0.23,0.43


In [67]:
cursor.execute("PRAGMA database_list;")
cursor.fetchall()


[(0, 'main', 'D:\\Machine_learning_projects\\Churn_database.db')]

In [68]:
import os
import sqlite3

db_path = os.path.abspath("Churn_database.db")   
print("DATABASE PATH:", db_path)


DATABASE PATH: D:\Machine_learning_projects\Churn_database.db


In [69]:
import os

for file in os.listdir():
    if file.endswith("Churn_database.db"):
        print(file, os.path.abspath(file))


Churn_database.db D:\Machine_learning_projects\Churn_database.db


In [70]:
pd.read_sql("PRAGMA table_info(churn_data);", connection)


,cid,name,type,notnull,dflt_value,pk
0,0,customerID,TEXT,0,None,0
1,1,gender,TEXT,0,None,0
2,2,SeniorCitizen,INTEGER,0,None,0
3,3,Partner,TEXT,0,None,0
4,4,Dependents,TEXT,0,None,0
5,5,tenure,INTEGER,0,None,0
6,6,PhoneService,TEXT,0,None,0
7,7,MultipleLines,TEXT,0,None,0
8,8,InternetService,TEXT,0,None,0
9,9,OnlineSecurity,TEXT,0,None,0


In [71]:
cursor = connection.cursor()
cursor.execute("PRAGMA database_list;")
print(cursor.fetchall())


[(0, 'main', 'D:\\Machine_learning_projects\\Churn_database.db')]


In [73]:
processed_data.to_csv("churn_data.csv", index=False)

print("churn_data.csv downloaded successfully")


churn_data.csv downloaded successfully


In [74]:
connection.close()